In [1]:
pip install --upgrade google-cloud-bigquery[pandas]

Requirement already up-to-date: google-cloud-bigquery[pandas] in c:\users\lenovo\anaconda3\lib\site-packages (1.25.0)
Note: you may need to restart the kernel to use updated packages.


# [Data Engineer Test - Analytic Lab Works]


In [2]:
# Cargar librerias
import pandas as pd
import os
from google.cloud import bigquery
from google.oauth2 import service_account
from dotenv import load_dotenv
from google.cloud import bigquery
load_dotenv()


True

# Punto 1:
Download the dataset from https://www.datos.gov.co/Seguridad-y-Defensa/Homicidios-2018/pz7x-mkbs 

 a.	Use BigQuery to store and read the data.


In [3]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="./clave/Test-42df6148e13f.json"

In [4]:
client = bigquery.Client()
filename = './Datos/Homicidios_2018.csv'
dataset_id = 'tablahom'
table_id = 'tabla_homicidios'

dataset_ref = client.dataset(dataset_id)
table_ref = dataset_ref.table(table_id)
job_config = bigquery.LoadJobConfig()
job_config.schema = [
    bigquery.SchemaField("fecha", "STRING"),
    bigquery.SchemaField("departamento", "STRING"),
    bigquery.SchemaField("municipio", "STRING"),
    bigquery.SchemaField("dia", "STRING"),
    bigquery.SchemaField("hora", "STRING"),
    bigquery.SchemaField("barrio", "STRING"),
    bigquery.SchemaField("zona", "STRING"),
    bigquery.SchemaField("clase_de_sitio", "STRING"),
    bigquery.SchemaField("arma_empleada", "STRING"),
    bigquery.SchemaField("movil_agresor", "STRING"),
    bigquery.SchemaField("movil_victima", "STRING"),
    bigquery.SchemaField("edad", "INT64"),
    bigquery.SchemaField("sexo", "STRING"),
    bigquery.SchemaField("estado_civil", "STRING"),
    bigquery.SchemaField("pais_nacimiento", "STRING"),
    bigquery.SchemaField("clase_empleado", "STRING"),
    bigquery.SchemaField("profesion", "STRING"),
    bigquery.SchemaField("escolaridad", "STRING"),
    bigquery.SchemaField("codigo_dane", "INT64"),
    bigquery.SchemaField("cantidad", "INT64"),
]
job_config.source_format = bigquery.SourceFormat.CSV
job_config.skip_leading_rows = 1
with open(filename, "rb") as source_file:
    job = client.load_table_from_file(source_file, table_ref,
        location="US", job_config=job_config)

job.result()  # Waits for table load to complete.

print("Loaded {} rows into {}:{}.".format(job.output_rows, dataset_id, table_id))

Loaded 4966 rows into tablahom:tabla_homicidios.


#### Comentario:
Se cargaron los datos en la tabla tabla_homicidios en GCP

In [6]:
# Llamamos la tabla
client = bigquery.Client()
sql = """
SELECT * 
FROM `atomic-legacy-282703.tablahom.tabla_homicidios` 
"""

df = client.query(sql).to_dataframe()

# Punto 2:	
What data quality issues do you evidence? How could you solve them? Write some querys in SQL to explore the data in Bigquery.

In [10]:
df[:2]

,fecha,departamento,municipio,dia,hora,barrio,zona,clase_de_sitio,arma_empleada,movil_agresor,movil_victima,edad,sexo,estado_civil,pais_nacimiento,clase_empleado,profesion,escolaridad,codigo_dane,cantidad
0,None,None,None,None,None,None,None,None,None,None,None,NaN,None,None,None,None,None,None,NaN,12667
1,11/02/2018 12:00:00 AM,CASANARE,VILLANUEVA,Domingo,2:00:00,LA COLMENA,RURAL,BILLARES,ARMA DE FUEGO,A PIE,A PIE,13.0,FEMENINO,SOLTERO,COLOMBIA,ESTUDIANTE,-,SECUNDARIA,85440000.0,1


In [14]:
sql = """
DELETE 
FROM `atomic-legacy-282703.tablahom.tabla_homicidios`
WHERE FECHA IS NULL AND DEPARTAMENTO IS NULL AND MUNICIPIO IS NULL 
"""
df = client.query(sql).to_dataframe()

#### Comentario: 
Eliminar el primer registro dado que no es representativo de la información 

In [15]:
# Top de 10 de departamentos - homicidios
sql = """
SELECT DEPARTAMENTO, COUNT(*) AS TOTAL_H_DPTO
FROM `atomic-legacy-282703.tablahom.tabla_homicidios`
GROUP BY 1
ORDER BY 2 DESC limit 10
"""

consulta1 = client.query(sql).to_dataframe()
consulta1

,DEPARTAMENTO,TOTAL_H_DPTO
0,ANTIOQUIA,926
1,VALLE,889
2,CUNDINAMARCA,581
3,CAUCA,282
4,NARIÑO,263
5,ATLÁNTICO,215
6,NORTE DE SANTANDER,181
7,BOLÍVAR,156
8,CÓRDOBA,139
9,META,116


#### Comentario: 
Se observa que los departamentos de Antioquia, Valle y Cundinamarca son los que presentan mayor número de homicidios

In [17]:
# Móvil agresor
sql = """
SELECT movil_agresor, COUNT(*) AS TOTAL_MA
FROM `atomic-legacy-282703.tablahom.tabla_homicidios`
WHERE (movil_agresor IS NOT NULL AND movil_agresor<>'-') AND (movil_victima IS NOT NULL AND movil_victima<>'-')
GROUP BY 1
ORDER BY 2 DESC
"""

consulta2 = client.query(sql).to_dataframe()
consulta2

,movil_agresor,TOTAL_MA
0,A PIE,4122
1,PASAJERO MOTOCICLETA,618
2,CONDUCTOR MOTOCICLETA,107
3,PASAJERO VEHICULO,29
4,PASAJERO TAXI,19
5,CONDUCTOR VEHICULO,18
6,BICICLETA,16
7,PASAJERO BARCO,7
8,CONDUCTOR TAXI,7
9,PASAJERO BUS,3


#### Comentario: 
La mayoría de personas que cometen un homicidio andan a pie o en moto. Se puede observar que es poco usual tomar transporte público para cometer un homicidio.

In [22]:
# Profesión
sql = """
SELECT profesion, (Count(profesion)* 100 / (Select Count(*) FROM  `atomic-legacy-282703.tablahom.tabla_homicidios`)) as Per
from  `atomic-legacy-282703.tablahom.tabla_homicidios`
group by 1  
order by 2 DESC limit 10
"""

consulta3 = client.query(sql).to_dataframe()
consulta3


,profesion,Per
0,-,97.442095
1,NO REPORTADA,0.946626
2,TECNICO PROFESIONAL EN SERVICIO DE POLICIA,0.483384
3,POLICIAS,0.261833
4,DERECHO,0.080564
5,LICENCIADO,0.080564
6,INGENIERO,0.080564
7,CONTADURIA PUBLICA,0.040282
8,TECNICO PROFESIONAL EN ADMINISTRACION DE EMPR...,0.040282
9,ADMINISTRACION DE EMPRESAS,0.040282


#### Comentario: 
Respecto a la variable profesion, se observa que un aprox 98% no tiene profesión y/o no reporta. La variable podría considerarse como flag o excluirse dado que es similar a una constante. 

Por otro lado, se observa que se ha utilizado ("-" y "NO REPORTADA"), Debe hacerse una transformación para que en las variables esté adecuadamente identificados los valores especiales y valores faltantes. 

In [23]:
# Edad
sql = """
SELECT MIN(EDAD) AS MINIMO, MAX(EDAD) AS MAXIMO, AVG(EDAD) AS PROMEDIO
FROM `atomic-legacy-282703.tablahom.tabla_homicidios`
"""

consulta4 = client.query(sql).to_dataframe()
consulta4

,MINIMO,MAXIMO,PROMEDIO
0,1,100,32.758711


#### Comentario: 
El promedio de las victimas de homicidios es 32.75 años de edad. El rando de edad de las victimas es de cero a cien.

In [25]:
# Género
sql = """
SELECT DISTINCT SEXO, 
COUNT(*) as TOTAL,
(Count(SEXO)* 100 / (Select Count(*) FROM  `atomic-legacy-282703.tablahom.tabla_homicidios`)) as Per
FROM `atomic-legacy-282703.tablahom.tabla_ref`
GROUP BY 1
order by 3
"""
consulta5 = client.query(sql).to_dataframe()
consulta5

,SEXO,TOTAL,Per
0,FEMENINO,410,8.257805
1,MASCULINO,4555,91.742195


#### Comentario: 
Del total de victimas de homicidios, el 91.7% son personas de género masculino y el 8.25% corresponden a personas de género femenino. 